In [21]:
import json
from IPython.display import GeoJSON
import pandas as pd
import geopandas as gpd
import folium
import urllib.parse
import requests
import os
import random
from datetime import datetime, timedelta

# Supress certificate warnings.
from urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(category=InsecureRequestWarning)

# CONSTANT: The Base URL for our API.
api_url = "https://taani-pub-ship.taani.oregonstate.edu/api"

# DERIVATIVES: 
sensor_url = api_url + "/sensor"
parameter_url = api_url + "/parameter"

# KNOWNS:
sensor_short_name = "Thermosalinograph"
parameter_short_name = "Temperature"
hour_interval = 1

In [53]:
sensor_float_url = "https://taani-pub-ship.taani.oregonstate.edu/api/sensor_float_1/"

# Set the start date and time using the ISO8601 format, data stored in UTC
end_date = datetime.now()
print(end_date)
start_date = end_date - timedelta(hours=0, minutes=60, seconds=0)
print(start_date)

# % encode the times
ed=urllib.parse.quote(str(end_date), safe='')
sd=urllib.parse.quote(str(start_date), safe='')
print(sd)
print(ed)

query_url = sensor_float_url+'?date_after='+sd+'&date_before='+ed+'&format=json'

print()
print(query_url)

# Make the query to the REST API
response = requests.get(query_url, verify=False)

# Load the response as json data
responseJSON = json.loads(response.text)

#print(responseJSON)
#print(json.dumps(responseJSON, indent=4, sort_keys=True))

2024-10-02 12:00:13.341642
2024-10-02 11:00:13.341642
2024-10-02%2011%3A00%3A13.341642
2024-10-02%2012%3A00%3A13.341642

https://taani-pub-ship.taani.oregonstate.edu/api/sensor_float_1/?date_after=2024-10-02%2011%3A00%3A13.341642&date_before=2024-10-02%2012%3A00%3A13.341642&format=json


In [50]:
# Step 3. Now that we know the sensor_id for an online/active Thermosalinograph, we want to know where to look to find the data.
# We'll need to find 3 additional peices of information before we can make a data request.  We need to know what table the data is stored in 
# (data_table), and we need to know what field in that table the temperature data is in (data_fieldname).  We also want the units for the parameter.

query_url = parameter_url+"/?sensor_id=seatsg450712&short_name=Temperature&format=json"
response = requests.get(query_url, verify=False)

# Load the response as json data
responseJSON = json.loads(response.text)
print(responseJSON)

# Create variables to store table and field information
data_table = responseJSON[0]['binned_default_table'][:-8]
data_field = responseJSON[0]['binned_default_fieldname']
data_units = responseJSON[0]['units_abbrev']

print("Thermosalinograph data is stored in table: " + data_table)
print("Thermosalinograph temperature data is stored in field: " + data_field)

[{'parameter_id': '9ded3f5f-6e16-456e-b991-fe3cc8ed7a44', 'short_name': 'Temperature', 'long_name': 'Temperature', 'description': 'Located at the Main Sensor Manifold, this thermosalinograph measures flowthrough sea surface temperature ranging from -5 to 35 degrees Celsius.', 'samos_data_category': 'Sea Surface Temperature', 'processing_status': 'raw', 'processing_symbol': 'temperature', 'diagnostic': False, 'gross_min': None, 'gross_max': None, 'recommended_min': None, 'recommended_max': None, 'global_min': -5.0, 'global_max': 35.0, 'local_min': None, 'local_max': None, 'detection_limit': None, 'saturated': None, 'bad_data_value': 9999.999, 'accuracy': 0.002, 'precision': None, 'resolution': 0.0001, 'data_model': 'SensorFloat1', 'data_table': 'sensor_float_1', 'archive_data_table': 'sensor_float_1_archive', 'data_fieldname': 'p1', 'flag_fieldname': 'f1', 'binned_default_model': 'BinnedDefaultFlowRolling', 'binned_default_table': 'binned_default_flow_rolling', 'binned_default_fieldname

In [92]:
# Step 4. Now that we know what table and field the data is in, we can make a data query.
# Data queries may be filtered by time.  You can select all data before a specific datetime, after a specific datetime, or within a datetime range.
# All dates must be formatted according to ISO8601

# Start building a query URL
data_query_url = api_url+'/'+data_table+'/'

# All datetimes are UTC, so get the current UTC time and take off one second
time_now = datetime.utcnow()
query_time = time_now - timedelta(minutes=10)

# % encode the space between date and time in the query_time
query_time=urllib.parse.quote(str(query_time), safe='')

query_url = data_query_url+'?date_after='+str(query_time)+'&format=json'
print(query_url)
response = requests.get(query_url, verify=False)

# Load the response as json data
responseJSON = json.loads(response.text)

m = folium.Map(location=(56.7, -146.6))

# Make coordinate list for polyline
line_coords = []

for record in responseJSON:
    X = record['longitude']
    Y = record['latitude']
    T = record[data_field]['a'][0]
    t = record['time']

    coordinates = (Y,X)
    line_coords.append(coordinates)

    # Add Points
    folium.Marker(
    location=[Y, X],
    tooltip="Time: " + t,
    popup="Temperature: " + str(T),
    icon=folium.Icon(icon="cloud"),
    ).add_to(m)

folium.PolyLine(line_coords).add_to(m)

m

https://taani-pub-ship.taani.oregonstate.edu/api/binned_default_flow/?date_after=2024-10-02%2019%3A39%3A22.543631&format=json
